In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

pd.options.display.max_rows = 100

In [2]:
import wrangle_zillow as w

## Acquire:

In [3]:
df = w.acquire_zillow()

Using cached csv


In [4]:
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1727539,14297519,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,6.059063e+13,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,1387261,17052889,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,6.111001e+13,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,11677,14186244,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,6.059022e+13,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,2288172,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,6.037300e+13,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,1970746,10887214,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,6.037124e+13,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN


In [5]:
df.shape

(77380, 68)

## Prepare:

In [6]:
# Identify and remove the non-single units
not_single = [246, 248, 247, 267, 31]
df = df[~df.propertylandusetypeid.isin(not_single)]

In [7]:
df.shape

(74065, 68)

In [8]:
# Take a look at what our nulls look like
col_nulls = w.nulls_by_col(df)

In [9]:
w.nulls_by_rows(df)

,num_cols_missing,num_rows,pct_cols_missing
0,23,2,0.338235
1,24,13,0.352941
2,25,24,0.367647
3,26,65,0.382353
4,27,316,0.397059
5,28,454,0.411765
6,29,5263,0.426471
7,30,3443,0.441176
8,31,9789,0.455882
9,32,12479,0.470588


In [10]:
# Investigating the columns that would be dropped by a 50% proportion requirement
poss_imputes = col_nulls.index[col_nulls.pct_rows_null > .49]
for col in poss_imputes:
    print(col)
    print(df[col].value_counts(dropna=False))
    print('----------')

airconditioningtypeid
NaN     49335
1.0     22943
13.0     1567
5.0       167
11.0       53
Name: airconditioningtypeid, dtype: int64
----------
architecturalstyletypeid
NaN     73859
7.0       172
8.0        19
21.0        7
2.0         5
3.0         3
Name: architecturalstyletypeid, dtype: int64
----------
basementsqft
NaN       74015
900.0         2
700.0         2
100.0         2
640.0         2
515.0         2
273.0         2
912.0         2
588.0         1
819.0         1
1809.0        1
604.0         1
126.0         1
669.0         1
314.0         1
800.0         1
786.0         1
396.0         1
112.0         1
645.0         1
384.0         1
1969.0        1
252.0         1
600.0         1
1218.0        1
280.0         1
3112.0        1
1416.0        1
224.0         1
512.0         1
380.0         1
204.0         1
200.0         1
90.0          1
405.0         1
300.0         1
674.0         1
1252.0        1
254.0         1
3560.0        1
352.0         1
168.0         1
394.0

1. fireplacecnt - all NaN can be 0, not encoded
3. hashottuborspa - all NaN can be 0, encode
4. poolcnt - all NaN can be 0, encode
5. threequarterbathnbr - NaN can be 0, not encoded
6. taxdelinquiencyflag - NaN can be No, can change to 1/0 encoded

The other columns are not worth 'saving' due to redundant information, inability/unnecessary to impute, or too many unexplained nulls

In [11]:
cols = ['fireplacecnt', 'hashottuborspa', 'poolcnt', 'threequarterbathnbr', 'taxdelinquencyflag']

In [12]:
for col in cols:
    df[col] = df[col].fillna(value=0)

In [13]:
for col in cols:
    print(df[col].value_counts(dropna=False))

0.0    65804
1.0     6996
2.0      994
3.0      233
4.0       35
5.0        3
Name: fireplacecnt, dtype: int64
0.0    72528
1.0     1537
Name: hashottuborspa, dtype: int64
0.0    57973
1.0    16092
Name: poolcnt, dtype: int64
0.0    63972
1.0    10009
2.0       75
3.0        8
7.0        1
Name: threequarterbathnbr, dtype: int64
0    71417
Y     2648
Name: taxdelinquencyflag, dtype: int64


In [14]:
# Dropping our columns that do not have 50% threshold for column or row of non-nulls
df = w.handle_missing_values(df, .5, .5)

In [15]:
df.shape

(73668, 39)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73668 entries, 0 to 77379
Data columns (total 39 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            73668 non-null  int64  
 1   parcelid                      73668 non-null  int64  
 2   bathroomcnt                   73668 non-null  float64
 3   bedroomcnt                    73668 non-null  float64
 4   buildingqualitytypeid         46900 non-null  float64
 5   calculatedbathnbr             73603 non-null  float64
 6   calculatedfinishedsquarefeet  73664 non-null  float64
 7   finishedsquarefeet12          73465 non-null  float64
 8   fips                          73668 non-null  float64
 9   fireplacecnt                  73668 non-null  float64
 10  fullbathcnt                   73603 non-null  float64
 11  hashottuborspa                73668 non-null  float64
 12  heatingorsystemtypeid         48505 non-null  float64
 13  l

In [17]:
# Rows to drop bc they are not useful, redundant, or cause leakage
to_drop = ['id', 'parcelid', 'calculatedbathnbr', 'finishedsquarefeet12', 'lotsizesquarefeet', 'propertycountylandusecode',
          'propertylandusetypeid', 'propertyzoningdesc', 'rawcensustractandblock', 'structuretaxvaluedollarcnt', 'taxvaluedollarcnt',
          'assessmentyear', 'landtaxvaluedollarcnt', 'taxamount', 'censustractandblock', 'transactiondate', 'heatingorsystemdesc', 
          'propertylandusedesc', 'buildingqualitytypeid', 'heatingorsystemtypeid', 'regionidcity', 'roomcnt', 'fullbathcnt',
          'regionidcounty']

In [18]:
df = df.drop(columns=to_drop)

In [19]:
df.shape

(73668, 15)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73668 entries, 0 to 77379
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bathroomcnt                   73668 non-null  float64
 1   bedroomcnt                    73668 non-null  float64
 2   calculatedfinishedsquarefeet  73664 non-null  float64
 3   fips                          73668 non-null  float64
 4   fireplacecnt                  73668 non-null  float64
 5   hashottuborspa                73668 non-null  float64
 6   latitude                      73668 non-null  float64
 7   longitude                     73668 non-null  float64
 8   poolcnt                       73668 non-null  float64
 9   regionidzip                   73629 non-null  float64
 10  threequarterbathnbr           73668 non-null  float64
 11  unitcnt                       47268 non-null  float64
 12  yearbuilt                     73629 non-null  float64
 13  t

In [21]:
df.unitcnt.value_counts(dropna=False)

1.0    47229
NaN    26400
2.0       33
3.0        3
4.0        2
6.0        1
Name: unitcnt, dtype: int64

In [22]:
df.unitcnt = df.unitcnt.fillna(value=1)

In [23]:
df.shape

(73668, 15)

In [24]:
df.unitcnt.value_counts(dropna=False)

1.0    73629
2.0       33
3.0        3
4.0        2
6.0        1
Name: unitcnt, dtype: int64

In [25]:
df = df[df.unitcnt <= 1]

In [26]:
df.shape

(73629, 15)

In [27]:
df = df.drop(columns='unitcnt')

In [28]:
df.shape

(73629, 14)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73629 entries, 0 to 77379
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bathroomcnt                   73629 non-null  float64
 1   bedroomcnt                    73629 non-null  float64
 2   calculatedfinishedsquarefeet  73625 non-null  float64
 3   fips                          73629 non-null  float64
 4   fireplacecnt                  73629 non-null  float64
 5   hashottuborspa                73629 non-null  float64
 6   latitude                      73629 non-null  float64
 7   longitude                     73629 non-null  float64
 8   poolcnt                       73629 non-null  float64
 9   regionidzip                   73590 non-null  float64
 10  threequarterbathnbr           73629 non-null  float64
 11  yearbuilt                     73590 non-null  float64
 12  taxdelinquencyflag            73629 non-null  object 
 13  l

In [30]:
w.nulls_by_col(df)

,count_nulls,pct_rows_null
bathroomcnt,0,0.000000
bedroomcnt,0,0.000000
calculatedfinishedsquarefeet,4,0.000054
fips,0,0.000000
fireplacecnt,0,0.000000
hashottuborspa,0,0.000000
latitude,0,0.000000
longitude,0,0.000000
poolcnt,0,0.000000
regionidzip,39,0.000530


In [31]:
w.nulls_by_rows(df)

,num_cols_missing,num_rows,pct_cols_missing
0,0,73551,0.000000
1,1,74,0.071429
2,2,4,0.142857


In [32]:
# Due to low number of nulls (would result in a .1% drop of data) and the desire to retain the integrity of 
# the columns where they are missing, am going to just drop all the nulls. 
df = df.dropna()

In [33]:
df.shape

(73551, 14)

In [34]:
# Changing yearbuilt to new feature age and dropping yearbuilt column
df.yearbuilt = df.yearbuilt.astype(int)
df['age'] = 2017 - df.yearbuilt
df = df.drop(columns='yearbuilt')

In [35]:
df.head()

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,fireplacecnt,hashottuborspa,latitude,longitude,poolcnt,regionidzip,threequarterbathnbr,taxdelinquencyflag,logerror,age
0,3.5,4.0,3100.0,6059.0,0.0,0.0,33634931.0,-117869207.0,0.0,96978.0,1.0,0,0.025595,19
1,1.0,2.0,1465.0,6111.0,1.0,0.0,34449266.0,-119281531.0,0.0,97099.0,0.0,0,0.055619,50
2,2.0,3.0,1243.0,6059.0,0.0,0.0,33886168.0,-117823170.0,1.0,97078.0,0.0,0,0.005383,55
3,3.0,4.0,2376.0,6037.0,0.0,0.0,34245180.0,-118240722.0,1.0,96330.0,0.0,0,-0.103410,47
4,3.0,3.0,1312.0,6037.0,0.0,0.0,34185120.0,-118414640.0,1.0,96451.0,0.0,0,0.006940,53


In [36]:
df.regionidzip.value_counts(dropna=False)

97319.0    818
96987.0    750
97318.0    692
97118.0    610
96974.0    590
          ... 
96500.0      2
96329.0      2
97092.0      1
96467.0      1
97111.0      1
Name: regionidzip, Length: 388, dtype: int64

In [37]:
df.bathroomcnt.value_counts(dropna=False)

2.0     30166
3.0     16599
1.0     12716
2.5      6583
4.0      2616
1.5      1405
3.5      1027
5.0       902
4.5       688
6.0       341
5.5       224
7.0        92
8.0        55
0.0        50
6.5        47
7.5        16
9.0        13
10.0        5
11.0        3
8.5         2
13.0        1
Name: bathroomcnt, dtype: int64

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73551 entries, 0 to 77379
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bathroomcnt                   73551 non-null  float64
 1   bedroomcnt                    73551 non-null  float64
 2   calculatedfinishedsquarefeet  73551 non-null  float64
 3   fips                          73551 non-null  float64
 4   fireplacecnt                  73551 non-null  float64
 5   hashottuborspa                73551 non-null  float64
 6   latitude                      73551 non-null  float64
 7   longitude                     73551 non-null  float64
 8   poolcnt                       73551 non-null  float64
 9   regionidzip                   73551 non-null  float64
 10  threequarterbathnbr           73551 non-null  float64
 11  taxdelinquencyflag            73551 non-null  object 
 12  logerror                      73551 non-null  float64
 13  a

In [39]:
df.taxdelinquencyflag.value_counts()

0    70913
Y     2638
Name: taxdelinquencyflag, dtype: int64

In [40]:
df.taxdelinquencyflag = np.where(df.taxdelinquencyflag == 'Y', 1, 0)

In [41]:
df.taxdelinquencyflag.value_counts()

0    70913
1     2638
Name: taxdelinquencyflag, dtype: int64

In [42]:
df.hashottuborspa.value_counts()

0.0    72014
1.0     1537
Name: hashottuborspa, dtype: int64

In [43]:
df.fireplacecnt.value_counts()

0.0    65316
1.0     6980
2.0      986
3.0      231
4.0       35
5.0        3
Name: fireplacecnt, dtype: int64

In [44]:
df.bedroomcnt.value_counts()

3.0     29817
2.0     18757
4.0     16528
5.0      4116
1.0      3264
6.0       647
0.0       281
7.0       107
8.0        23
9.0         8
10.0        2
11.0        1
Name: bedroomcnt, dtype: int64

In [45]:
df.bathroomcnt.value_counts()

2.0     30166
3.0     16599
1.0     12716
2.5      6583
4.0      2616
1.5      1405
3.5      1027
5.0       902
4.5       688
6.0       341
5.5       224
7.0        92
8.0        55
0.0        50
6.5        47
7.5        16
9.0        13
10.0        5
11.0        3
8.5         2
13.0        1
Name: bathroomcnt, dtype: int64

In [46]:
df.calculatedfinishedsquarefeet

0        3100.0
1        1465.0
2        1243.0
3        2376.0
4        1312.0
          ...  
77375    1741.0
77376    1286.0
77377    1612.0
77378    1032.0
77379    1762.0
Name: calculatedfinishedsquarefeet, Length: 73551, dtype: float64

In [47]:
df = df[(df.bathroomcnt > 0) & (df.bedroomcnt > 0) & (df.calculatedfinishedsquarefeet > 400)]

In [48]:
df.shape

(73236, 14)

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73236 entries, 0 to 77379
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bathroomcnt                   73236 non-null  float64
 1   bedroomcnt                    73236 non-null  float64
 2   calculatedfinishedsquarefeet  73236 non-null  float64
 3   fips                          73236 non-null  float64
 4   fireplacecnt                  73236 non-null  float64
 5   hashottuborspa                73236 non-null  float64
 6   latitude                      73236 non-null  float64
 7   longitude                     73236 non-null  float64
 8   poolcnt                       73236 non-null  float64
 9   regionidzip                   73236 non-null  float64
 10  threequarterbathnbr           73236 non-null  float64
 11  taxdelinquencyflag            73236 non-null  int64  
 12  logerror                      73236 non-null  float64
 13  a

In [50]:
# Fips has to be converted to int, str, and concat '0' to properly format number code
df.fips = df.fips.astype(int)
df.fips = df.fips.astype(str)
df.fips = '0' + df.fips

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73236 entries, 0 to 77379
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bathroomcnt                   73236 non-null  float64
 1   bedroomcnt                    73236 non-null  float64
 2   calculatedfinishedsquarefeet  73236 non-null  float64
 3   fips                          73236 non-null  object 
 4   fireplacecnt                  73236 non-null  float64
 5   hashottuborspa                73236 non-null  float64
 6   latitude                      73236 non-null  float64
 7   longitude                     73236 non-null  float64
 8   poolcnt                       73236 non-null  float64
 9   regionidzip                   73236 non-null  float64
 10  threequarterbathnbr           73236 non-null  float64
 11  taxdelinquencyflag            73236 non-null  int64  
 12  logerror                      73236 non-null  float64
 13  a

In [52]:
df.age.value_counts()

62     1907
28     1761
67     1604
53     1558
63     1530
       ... 
125       1
139       1
135       1
123       1
137       1
Name: age, Length: 133, dtype: int64

In [53]:
outlier_check = ['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'age']

In [54]:
df = w.remove_outliers(df, 1.5, outlier_check)

In [55]:
df.shape

(68020, 14)

In [56]:
df.threequarterbathnbr.value_counts()

0.0    59521
1.0     8479
2.0       20
Name: threequarterbathnbr, dtype: int64

In [57]:
df.regionidzip.value_counts().head(50)

97319.0    743
97318.0    682
96987.0    662
96993.0    574
96974.0    574
97328.0    571
97118.0    552
97329.0    522
96962.0    502
96996.0    473
97116.0    470
96193.0    460
96368.0    451
96995.0    448
96964.0    419
96186.0    396
96505.0    387
97078.0    384
96389.0    384
96364.0    377
96378.0    369
96373.0    368
96998.0    365
96370.0    364
97317.0    361
96351.0    361
96385.0    355
97083.0    353
96236.0    352
97091.0    351
96954.0    346
97330.0    346
96377.0    345
97089.0    341
97041.0    335
97068.0    334
96401.0    332
96383.0    328
96963.0    321
96242.0    320
96190.0    319
96124.0    315
96122.0    312
97026.0    303
97008.0    302
96990.0    302
96966.0    300
97023.0    295
96369.0    295
96989.0    295
Name: regionidzip, dtype: int64

In [58]:
df.fips.value_counts()

06037    44100
06059    18330
06111     5590
Name: fips, dtype: int64

In [59]:
df.poolcnt = df.poolcnt.astype(int)
df.hashottuborspa = df.hashottuborspa.astype(int)
df.regionidzip = df.regionidzip.astype(int)
cols = ['fips', 'hashottuborspa', 'poolcnt', 'taxdelinquencyflag', 'regionidzip']

In [60]:
for col in cols:
    df[col] = df[col].astype(object)

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68020 entries, 0 to 77379
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bathroomcnt                   68020 non-null  float64
 1   bedroomcnt                    68020 non-null  float64
 2   calculatedfinishedsquarefeet  68020 non-null  float64
 3   fips                          68020 non-null  object 
 4   fireplacecnt                  68020 non-null  float64
 5   hashottuborspa                68020 non-null  object 
 6   latitude                      68020 non-null  float64
 7   longitude                     68020 non-null  float64
 8   poolcnt                       68020 non-null  object 
 9   regionidzip                   68020 non-null  object 
 10  threequarterbathnbr           68020 non-null  float64
 11  taxdelinquencyflag            68020 non-null  object 
 12  logerror                      68020 non-null  float64
 13  a

In [62]:
encode_cols = ['fips', 'hashottuborspa', 'poolcnt', 'taxdelinquencyflag']

In [63]:
for col in encode_cols:
    df = pd.get_dummies(df, columns=[col])

In [64]:
df.head()

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fireplacecnt,latitude,longitude,regionidzip,threequarterbathnbr,logerror,age,fips_06037,fips_06059,fips_06111,hashottuborspa_0,hashottuborspa_1,poolcnt_0,poolcnt_1,taxdelinquencyflag_0,taxdelinquencyflag_1
0,3.5,4.0,3100.0,0.0,33634931.0,-117869207.0,96978,1.0,0.025595,19,0,1,0,1,0,1,0,1,0
1,1.0,2.0,1465.0,1.0,34449266.0,-119281531.0,97099,0.0,0.055619,50,0,0,1,1,0,1,0,1,0
2,2.0,3.0,1243.0,0.0,33886168.0,-117823170.0,97078,0.0,0.005383,55,0,1,0,1,0,0,1,1,0
3,3.0,4.0,2376.0,0.0,34245180.0,-118240722.0,96330,0.0,-0.103410,47,1,0,0,1,0,0,1,1,0
4,3.0,3.0,1312.0,0.0,34185120.0,-118414640.0,96451,0.0,0.006940,53,1,0,0,1,0,0,1,1,0


In [65]:
df = df.drop(columns = ['hashottuborspa_0', 'poolcnt_0', 'taxdelinquencyflag_0'])

In [66]:
df.head()

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fireplacecnt,latitude,longitude,regionidzip,threequarterbathnbr,logerror,age,fips_06037,fips_06059,fips_06111,hashottuborspa_1,poolcnt_1,taxdelinquencyflag_1
0,3.5,4.0,3100.0,0.0,33634931.0,-117869207.0,96978,1.0,0.025595,19,0,1,0,0,0,0
1,1.0,2.0,1465.0,1.0,34449266.0,-119281531.0,97099,0.0,0.055619,50,0,0,1,0,0,0
2,2.0,3.0,1243.0,0.0,33886168.0,-117823170.0,97078,0.0,0.005383,55,0,1,0,0,1,0
3,3.0,4.0,2376.0,0.0,34245180.0,-118240722.0,96330,0.0,-0.103410,47,1,0,0,0,1,0
4,3.0,3.0,1312.0,0.0,34185120.0,-118414640.0,96451,0.0,0.006940,53,1,0,0,0,1,0


In [67]:
df = df.rename(columns={'bathroomcnt': 'bathrooms',
                       'bedroomcnt': 'bedrooms', 'calculatedfinishedsquarefeet': 'squarefeet', 'fireplacecnt': 'num_fireplace',
                       'threequarterbathnbr': 'threequarter_baths', 'hashottuborspa_1': 'hottub_or_spa', 'poolcnt_1': 'has_pool',
                       'taxdelinquencyflag_1': 'tax_delinquency'})

In [68]:
df.head()

,bathrooms,bedrooms,squarefeet,num_fireplace,latitude,longitude,regionidzip,threequarter_baths,logerror,age,fips_06037,fips_06059,fips_06111,hottub_or_spa,has_pool,tax_delinquency
0,3.5,4.0,3100.0,0.0,33634931.0,-117869207.0,96978,1.0,0.025595,19,0,1,0,0,0,0
1,1.0,2.0,1465.0,1.0,34449266.0,-119281531.0,97099,0.0,0.055619,50,0,0,1,0,0,0
2,2.0,3.0,1243.0,0.0,33886168.0,-117823170.0,97078,0.0,0.005383,55,0,1,0,0,1,0
3,3.0,4.0,2376.0,0.0,34245180.0,-118240722.0,96330,0.0,-0.103410,47,1,0,0,0,1,0
4,3.0,3.0,1312.0,0.0,34185120.0,-118414640.0,96451,0.0,0.006940,53,1,0,0,0,1,0


In [69]:
df.shape

(68020, 16)